In [4]:
from selenium import webdriver
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
import re
import time
import requests
import pandas
from bs4 import BeautifulSoup
import json
import re
import numpy
import itertools
import pandas as pd
from unidecode import unidecode
from time import sleep
from IPython.display import clear_output

In [5]:
url = 'https://www.hbogola.com/home'

In [14]:
driver = webdriver.Chrome("/usr/lib/chromium-browser/chromedriver")#Driver de crome
driver.get(url)# Aca utilizamos un get para el driver
driver.maximize_window()#maximizamos la ventana
cookies_xpath = '//*[@id="onetrust-accept-btn-handler"]'#Identificamos las cookies
WebDriverWait(driver, 10).until(EC.presence_of_element_located((By.XPATH, cookies_xpath))).click()#Aceptamos las cookies
kids_xpath = '//*[@id="root"]/div/div[1]/div/div[1]/div/ul/li[5]/a/span'
WebDriverWait(driver, 10).until(EC.presence_of_element_located((By.XPATH, kids_xpath))).click()
storeby_xpath = '//*[@id="root"]/div/div[2]/div/div/div[2]/div/div/div/span'
WebDriverWait(driver, 10).until(EC.presence_of_element_located((By.XPATH, storeby_xpath))).click()
az_xpath = '//*[@id="root"]/div/div[2]/div/div/div[2]/div/div/div[3]/div[1]/span'
WebDriverWait(driver, 10).until(EC.presence_of_element_located((By.XPATH, az_xpath))).click()

In [7]:
def get_kids_info(url):
    """
    Esta función recibe la url de una película infantil
    y devuelve un diccionario con la metadata asociada.
    """
    # Get de la url
    #driver.get(url)
    
    # Esperamos a que haya cargado la página
    WebDriverWait(driver, 5).until(lambda d: d.find_element_by_tag_name("p"))
    
    ## Buscamos los distintos elementos a partir de su class name
    # titleOfMovie
    title = driver.find_element(By.CLASS_NAME, "titleOfMovie")
    
    # detailsElement
    details = driver.find_elements(By.CLASS_NAME, "detailsElement")
    # Extraemos el contenido de cada elemento de details y normalizamos el texto
    details = [unidecode(detail.text) for detail in details]
    
    # abstract
    abstract = driver.find_element(By.CLASS_NAME, "abstract")
    
    # languagesAndSubtitlesContainer
    lang_sub = driver.find_element(By.CLASS_NAME, "languagesAndSubtitlesContainer")
    
    # castNCrewHeaderElement
    cast_crew_header = driver.find_elements(By.CLASS_NAME, "castNCrewHeaderElement")
    # Extraemos el texto de cada header
    headers = [h.text for h in cast_crew_header]
    
    # castNCrewBodyElement
    cast_crew_body = driver.find_elements(By.CLASS_NAME, "castNCrewBodyElement")
    # Extraemos el texto de cada body
    bodies = [b.text.split('\n') for b in cast_crew_body]
    
    # Zip de cast and crew
    cast_crew = dict(zip(headers, bodies))
    
    # Creamos un diccionario donde ir volcando la info de la película
    kids = dict()
    kids['title'] = title.text
    kids['details'] = details
    kids['languages_subs'] = lang_sub.text.split('\n')
    kids['cast_crew'] =  cast_crew
    
    return kids

In [8]:
kids_movies = list(); error_urls = list()

In [9]:
letters = ['#', 'A', 'B', 'C', 'D', 'E', 'F', 'G', 'H', 'I', 'J', 'K', 'L', 'M',
           'N', 'O', 'P', 'Q', 'R', 'S', 'T', 'U', 'V', 'W', 'X', 'Y', 'Z']
ids = list(range(3, len(letters)+3))
letters_ids = dict(zip(letters, ids))
letters_ids

{'#': 3,
 'A': 4,
 'B': 5,
 'C': 6,
 'D': 7,
 'E': 8,
 'F': 9,
 'G': 10,
 'H': 11,
 'I': 12,
 'J': 13,
 'K': 14,
 'L': 15,
 'M': 16,
 'N': 17,
 'O': 18,
 'P': 19,
 'Q': 20,
 'R': 21,
 'S': 22,
 'T': 23,
 'U': 24,
 'V': 25,
 'W': 26,
 'X': 27,
 'Y': 28,
 'Z': 29}

In [10]:
def get_kids_movies_by_letter_id(letter_id):
    x = letter_id
    y = 1
    z = 1
    while True:
        try:
            driver.find_element(By.XPATH, f'//*[@id="root"]/div/div[2]/div/div/div[{x}]/div/div/div/div[{y}]/div[{z}]/div/img').click()
            sleep(3)
            current_url = driver.current_url
            try:
                kids = get_kids_info(current_url)
                kids_movies.append(kids)
            except:
                print(f'Error en {current_url}')
                error_urls.append(current_url)
            driver.back()
            sleep(3)
            z += 1
        except:
            y += 1
            z = 1
            try:
                driver.find_element(By.XPATH, f'//*[@id="root"]/div/div[2]/div/div/div[{x}]/div/div/div/div[{y}]/div[{z}]/div/img').click()
                sleep(3)
                current_url = driver.current_url
                try:
                    kids = get_kids_info(current_url)
                    kids_movies.append(kids)
                except:
                    print(f'Error en {current_url}')
                    error_urls.append(current_url)
                driver.back()
                sleep(3)
                z += 1
            except:
                break

In [15]:
for letter_id in ids:
    try:
        get_kids_movies_by_letter_id(letter_id)
    except:
        pass

In [16]:
kids_movies

[{'title': 'An American Girl: Chrissa Stands Strong',
  'details': ['2009', '1h 30m', 'Adventure', 'Family', 'PG'],
  'languages_subs': ['Languages: Original, Spanish, Portuguese',
   'Subtitles: Spanish, Portuguese'],
  'cast_crew': {'Director': ['Martha Coolidge'],
   'Cast': ['Tara James',
    'Nana Louise Hanlon',
    'Tyler Maxwell',
    'Chrissa Maxwell',
    'Sonali Matthews'],
   '': ['Adair Tishler',
    'Michael Learned',
    'Austin Thomas',
    'Sammi Hanratty',
    'Ariela Barer'],
   'Writer': ['Mary Casanova', 'Julie Prendiville Roux', 'Christine Coy']}},
 {'title': 'Animals United',
  'details': ['2010', '1h 33m', 'COMEDIA', 'COMEDIA', 'G'],
  'languages_subs': ['Languages: Original, Spanish, Portuguese',
   'Subtitles: Spanish, Portuguese, English'],
  'cast_crew': {'Director': ['Tappe Holger', 'Reinhard Klooss'],
   'Cast': ['Socrates', 'Escrito por', 'Winifred'],
   '': ['Stephen Fry', 'Sven Severin', 'Vanessa Redgrave'],
   'Writer': ['Oliver Huzly']}},
 {'title': '

In [17]:
with open('kids_movies.json', 'w') as file:
    json.dump(kids_movies, file)